In [18]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import os
import gc

In [19]:
def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i][1] >= array[begin][1]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

In [11]:
TargetClass='Pvalue'
main_directory= '/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches'
main_save_folder="/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/FeatureSelection/T-test"
#directory='/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches/censored_CTNNB1_Target'
if not os.path.exists(main_directory):
    print("Bad routing.")
main_data={}
for folder in os.listdir(main_directory):
    directory=main_directory+'/'+folder
    save_folder=main_save_folder+'/'+folder
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    main_data[folder]=[]
    for filename in os.listdir(directory):
        preliminary_results=[]
        data = pd.read_csv(directory+'/'+filename)
        data['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1),data['Pvalue']))
        data.pop('Unnamed: 0')
        data.pop('TrueIndex')
        data.pop('CauseGene')
        data.pop('EffectGene')
        data.pop('Replicate')
        data.pop('Treatment')
        preliminary_results=[]
        for key in data:
            if key!=TargetClass:
                result=ttest_ind(data[TargetClass],data[key],equal_var)
                preliminary_results.append([key,result[1]])
        quicksort(preliminary_results)
        # write out
        df = pd.DataFrame(preliminary_results)
        df.to_csv(save_folder+"/results_"+filename)
        main_data[folder].append(preliminary_results)
        del df
        del data
        gc.collect()
# process main_data
# top 100
for key in main_data:
    for data in main_data[key]:
        data=data[:100]


AttributeError: module 'pandas' has no attribute 'dataFrame'

In [22]:
for key in main_data:
    print(key)
    for data in main_data[key][:1]:
        print(data[:10])


censored_Erk_Target
[['C15_C90~E240_E360', 0.98829021563499297], ['C0_C15~E210_E240', 0.98741390902722803], ['C30_C90~E90_E210', 0.97587411157702764], ['C0_C15~E15_E360', 0.97520765882049953], ['C15_C180~E30_E210', 0.96389508970221538], ['C0_C180~E90_E180', 0.95229116689179916], ['C15_C120~E30_E210', 0.93998763312405098], ['C30_C90~E180_E210', 0.93800556977457128], ['C210_C360~E240_E360', 0.93794095507985586], ['C90_C360~E240_E360', 0.91834436366121097]]
censored_LEF1_Target
[['C0_C15~E30_E360', 1.0], ['C0_C120~E15_E360', 0.98816762996368324], ['C120_C360~E120_E360', 0.98801912867366415], ['C15_C210~E30_E210', 0.98790970154599878], ['C30_C120~E30_E120', 0.98740495810193418], ['C30_C360~E210_E360', 0.97668135706989667], ['C120_C240~E180_E360', 0.97518451664541028], ['C15_C90~E30_E240', 0.96538573706182196], ['C0_C180~E90_E240', 0.96439530768054627], ['C210_C360~E240_E360', 0.96312486880355785]]
censored_IRF4_Target
[['C0_C210~E0_E240', 1.0], ['C0_C90~E210_E360', 0.98791894277850212], ['

In [42]:
condensed_results={}
top_rank=10
for key in main_data:
    # per appearance in top_rank
    condensed_results[key]={}
    for j  in range(0,len(main_data[key][0])):
        condensed_results[key][main_data[key][0][j][0]]=0

for key in main_data:
    for sub_data in main_data[key]:
        for data in sub_data:
            if sub_data.index(data)<top_rank:
                condensed_results[key][data[0]]+=1
        

In [24]:
main_data['censored_Erk_Target'][0][0][1]

0.98829021563499297

In [26]:
condensed_results

{'censored_CTNNB1_Target': {'C0_C120~E0_E120': 0,
  'C0_C120~E0_E180': 0,
  'C0_C120~E0_E210': 0,
  'C0_C120~E0_E240': 0,
  'C0_C120~E0_E360': 0,
  'C0_C120~E120_E180': 0,
  'C0_C120~E120_E210': 0,
  'C0_C120~E120_E240': 0,
  'C0_C120~E120_E360': 0,
  'C0_C120~E15_E120': 0,
  'C0_C120~E15_E180': 0,
  'C0_C120~E15_E210': 0,
  'C0_C120~E15_E240': 0,
  'C0_C120~E15_E360': 0,
  'C0_C120~E180_E210': 0,
  'C0_C120~E180_E240': 0,
  'C0_C120~E180_E360': 0,
  'C0_C120~E210_E240': 0,
  'C0_C120~E210_E360': 0,
  'C0_C120~E240_E360': 0,
  'C0_C120~E30_E120': 0,
  'C0_C120~E30_E180': 0,
  'C0_C120~E30_E210': 0,
  'C0_C120~E30_E240': 0,
  'C0_C120~E30_E360': 0,
  'C0_C120~E90_E120': 0,
  'C0_C120~E90_E180': 0,
  'C0_C120~E90_E210': 0,
  'C0_C120~E90_E240': 0,
  'C0_C120~E90_E360': 0,
  'C0_C15~E0_E120': 0,
  'C0_C15~E0_E15': 0,
  'C0_C15~E0_E180': 0,
  'C0_C15~E0_E210': 0,
  'C0_C15~E0_E240': 0,
  'C0_C15~E0_E30': 0,
  'C0_C15~E0_E360': 0,
  'C0_C15~E0_E90': 0,
  'C0_C15~E120_E180': 0,
  'C0_C15~E12

In [55]:
final_results={}
print_results={}
for main_key in condensed_results:
    final_results[main_key]=[]
    for key in condensed_results[main_key]:
        final_results[main_key].append([key,condensed_results[main_key][key]])
    quicksort(final_results[main_key])
    print(main_key)
    print_results[main_key]=[]
    for i in range(0,top_rank):
        print(final_results[main_key][i][0])
        print_results[main_key].append(final_results[main_key][i][0])
    
df=pd.DataFrame.from_dict(print_results, orient="index")
df.to_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/FeatureSelection/T-test/top"+str(top_rank)+"_results.csv")


censored_Erk_Target
C0_C210~E15_E210
C180_C360~E240_E360
C15_C180~E15_E240
C0_C90~E15_E360
C30_C360~E240_E360
C15_C120~E30_E210
C15_C120~E30_E240
C30_C90~E180_E210
C15_C120~E15_E240
C15_C90~E210_E240
censored_LEF1_Target
C30_C360~E30_E360
C15_C90~E180_E210
C120_C360~E240_E360
C30_C240~E210_E240
C0_C90~E0_E180
C0_C240~E15_E360
C0_C15~E120_E240
C90_C360~E90_E360
C0_C180~E210_E240
C30_C180~E30_E180
censored_IRF4_Target
C0_C90~E15_E210
C15_C90~E30_E180
C90_C240~E120_E240
C90_C210~E210_E240
C90_C240~E120_E360
C0_C210~E15_E210
C240_C360~E240_E360
C30_C90~E90_E210
C15_C240~E15_E240
C30_C360~E30_E360
censored_CTNNB1_Target
C0_C15~E0_E360
C15_C240~E15_E360
C0_C15~E210_E360
C15_C90~E210_E240
C0_C90~E30_E180
C30_C360~E120_E360
C15_C180~E15_E360
C15_C90~E30_E240
C30_C240~E90_E360
C90_C210~E120_E240
censored_Ikk2_Target
C15_C210~E30_E210
C15_C120~E15_E210
C15_C30~E30_E180
C15_C360~E15_E360
C90_C240~E90_E240
C30_C360~E30_E360
C15_C240~E15_E360
C15_C240~E15_E240
C30_C240~E30_E240
C15_C90~E30_E180
cen

In [37]:
final_results

{'censored_CTNNB1_Target': [['C30_C90~E120_E360', 43.17129115965411],
  ['C15_C90~E120_E240', 42.915040952839675],
  ['C0_C210~E15_E210', 56.132490402496174],
  ['C0_C15~E0_E180', 44.644603341908379],
  ['C15_C90~E15_E240', 52.020528728541834],
  ['C0_C120~E0_E210', 54.214546102112422],
  ['C30_C90~E180_E360', 54.690597057435319],
  ['C15_C210~E120_E210', 57.286408455658517],
  ['C30_C90~E30_E240', 53.462249152711607],
  ['C15_C180~E15_E180', 47.996783847192603],
  ['C0_C180~E0_E240', 59.447225025686919],
  ['C0_C120~E30_E210', 56.752127948277987],
  ['C30_C240~E180_E240', 61.683276108164037],
  ['C15_C120~E15_E360', 57.830573345280122],
  ['C0_C15~E120_E360', 56.402211371973131],
  ['C15_C180~E30_E210', 45.488677386465355],
  ['C0_C15~E0_E360', 54.506543895956483],
  ['C0_C210~E0_E210', 48.690895894943253],
  ['C0_C210~E0_E240', 59.839446610886185],
  ['C0_C90~E120_E180', 46.40872340837776],
  ['C30_C360~E30_E360', 58.13969012246654],
  ['C0_C180~E120_E240', 45.766730416397131],
  ['C